In [1]:
# A 'magic' command to display plots inline
%matplotlib inline

# Import all python modules. You need to run this cell for the other cells to work.
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# point = 'POINT(-118.2437 34.0522)' #LA

point =  'POINT(-121.8677 37.9833)'  # San Joaquin Valley
location = 'San Joaquin Valley'


In [3]:
# Cal-Adapt API
api = 'http://api.cal-adapt.org/api'

# Name of boundary layer in API (Locagrid)
resource = 'locagrid'

#Get ID
url = api + '/' + resource + '/?intersects=' +  point

response = requests.get(url)
data = response.json()
data = data['features'][0]
id = data['id']
print(id)

39458


## Download Data

In [4]:
# Cal-Adapt API
api = 'http://api.cal-adapt.org/api'
# Request header
headers = {'ContentType': 'json'}

def process_daily_data(slug):
    # Make request
    url = api + '/series/' + slug + '/events/?ref=/api/locagrid/%s/&imperial=True&stat=mean' % id
    response = requests.get(url)
    #print(response)
    # Get data
    if response.ok:
        data = response.json()
        df = pd.DataFrame(data)
        df.set_index('index', inplace = True)
        df.index = pd.to_datetime(df.index)
    
    else:
        print('Failed to download:')
    return df

In [5]:
def get_projections():
    # Create an empty list to hold dataframes
    df_list = []

    # Make a combined list of GCMs, scenarios, climate variables for looping
    climvar = ['pr', 'tasmax']
    period = ['day']
    #models = ['CanESM2']
    models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5', 'ACCESS1-0', 'CCSM4', 'CESM1-BGC', 'CMCC-CMS', 'GFDL-CM3', 'HadGEM2-CC']
    #models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
    scenarios = ['historical', 'rcp85']
    zipped = itertools.product(climvar, period, models, scenarios)

    # Loop through zipped
    for item in zipped:
        # Create slug
        slug = '_'.join(item)
        df = process_daily_data(slug)
        df_list.append(df)
            
    # Combine all the dataframes into one and return
    return pd.concat(df_list)

In [6]:
projectedDF = get_projections()
#units: inches/day
projectedDF.head()

,name,data
index,,
1950-01-01 00:00:00+00:00,pr_day_CanESM2_historical,0.659182
1950-01-02 00:00:00+00:00,pr_day_CanESM2_historical,0.264758
1950-01-03 00:00:00+00:00,pr_day_CanESM2_historical,0.042206
1950-01-04 00:00:00+00:00,pr_day_CanESM2_historical,0.160164
1950-01-05 00:00:00+00:00,pr_day_CanESM2_historical,0.021967


In [7]:
# Keep only required parameters in slug
projectedDF['Climate Data'] = projectedDF['name'].str.split('_', expand=True)[0]
projectedDF['GCM'] = projectedDF['name'].str.split('_', expand=True)[2]
projectedDF.index.name = 'Date'
projectedDF.drop(columns= 'name', inplace = True)
projectedDF.head()

,data,Climate Data,GCM
Date,,,
1950-01-01 00:00:00+00:00,0.659182,pr,CanESM2
1950-01-02 00:00:00+00:00,0.264758,pr,CanESM2
1950-01-03 00:00:00+00:00,0.042206,pr,CanESM2
1950-01-04 00:00:00+00:00,0.160164,pr,CanESM2
1950-01-05 00:00:00+00:00,0.021967,pr,CanESM2


In [8]:
projectedDF = projectedDF.pivot_table( values = 'data', index = projectedDF.index, columns = ['Climate Data', 'GCM'])

In [9]:
projectedDF.head()

Climate Data                     pr                                          \
GCM                       ACCESS1-0     CCSM4 CESM1-BGC  CMCC-CMS  CNRM-CM5   
Date                                                                          
1950-01-01 00:00:00+00:00  0.032311  0.010045  0.030709  0.500307  0.015855   
1950-01-02 00:00:00+00:00  0.089549  0.014309  0.082504  0.159410  0.008238   
1950-01-03 00:00:00+00:00  0.722789  0.045712  0.023114  0.000000  0.009229   
1950-01-04 00:00:00+00:00  0.447062  0.023548  0.004315  0.000000  0.000168   
1950-01-05 00:00:00+00:00  0.229132  0.105878  0.000000  0.000110  0.000000   

Climate Data                                                                \
GCM                         CanESM2  GFDL-CM3 HadGEM2-CC HadGEM2-ES MIROC5   
Date                                                                         
1950-01-01 00:00:00+00:00  0.659182  0.000000   0.000000   0.000206    0.0   
1950-01-02 00:00:00+00:00  0.264758  0.000000   0.009731   0.000000    0.0   
1950-01-03 00:00:00+00:00  0.042206  0.001381   0.480124   0.011716    0.0   
1950-01-04 00:00:00+00:00  0.160164  0.090342   0.323959   0.000000    0.0   
1950-01-05 00:00:00+00:00  0.021967  0.000843   0.319965   0.000000    0.0   

Climate Data                  tasmax                                   \
GCM                        ACCESS1-0      CCSM4  CESM1-BGC   CMCC-CMS   
Date                                                                    
1950-01-01 00:00:00+00:00  54.161400  52.582886  57.369904  57.100021   
1950-01-02 00:00:00+00:00  57.349358  57.304207  58.465569  51.715736   
1950-01-03 00:00:00+00:00  60.840263  53.671959  58.243370  45.774548   
1950-01-04 00:00:00+00:00  56.377785  55.651695  58.438213  47.260010   
1950-01-05 00:00:00+00:00  55.296947  59.364693  52.217480  41.186546   

Climate Data                                                           \
GCM                         CNRM-CM5    CanESM2   GFDL-CM3 HadGEM2-CC   
Date                                                                    
1950-01-01 00:00:00+00:00  65.326645  59.927246  45.232979  54.752190   
1950-01-02 00:00:00+00:00  63.081421  62.757103  44.550945  46.655376   
1950-01-03 00:00:00+00:00  62.672344  57.619621  45.793610  53.948376   
1950-01-04 00:00:00+00:00  61.072571  58.939522  46.588909  55.525520   
1950-01-05 00:00:00+00:00  58.839653  48.802322  50.458183  54.587173   

Climate Data                                     
GCM                       HadGEM2-ES     MIROC5  
Date                                             
1950-01-01 00:00:00+00:00  54.921764  53.597748  
1950-01-02 00:00:00+00:00  47.753407  54.462921  
1950-01-03 00:00:00+00:00  52.154366  53.432625  
1950-01-04 00:00:00+00:00  52.648308  55.889771  
1950-01-05 00:00:00+00:00  53.474316  56.541973

In [10]:
precip = projectedDF.xs(('pr'), axis = 1)
precip = precip.apply(lambda x: x*25.4) 
tmax = projectedDF.xs(('tasmax'), axis = 1)
precip_waterYear = precip

In [11]:
precip.head()

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5
Date,,,,,,,,,,
1950-01-01 00:00:00+00:00,0.820687,0.255154,0.780021,12.707801,0.402716,16.743221,0.000000,0.000000,0.00524,0.0
1950-01-02 00:00:00+00:00,2.274546,0.363442,2.095610,4.049003,0.209240,6.724862,0.000000,0.247158,0.00000,0.0
1950-01-03 00:00:00+00:00,18.358842,1.161089,0.587093,0.000000,0.234423,1.072030,0.035088,12.195158,0.29759,0.0
1950-01-04 00:00:00+00:00,11.355387,0.598111,0.109596,0.000000,0.004274,4.068159,2.294689,8.228564,0.00000,0.0
1950-01-05 00:00:00+00:00,5.819963,2.689298,0.000000,0.002790,0.000000,0.557955,0.021407,8.127099,0.00000,0.0


In [12]:
models = precip.columns.values

In [13]:
models

array(['ACCESS1-0', 'CCSM4', 'CESM1-BGC', 'CMCC-CMS', 'CNRM-CM5',
       'CanESM2', 'GFDL-CM3', 'HadGEM2-CC', 'HadGEM2-ES', 'MIROC5'],
      dtype=object)

#### Rearrange data to find Top Tercile prior year Oct to Apr 

In [14]:
precip_waterYear['year'] = precip_waterYear.index.year
precip_waterYear['month'] = precip_waterYear.index.month

In [15]:
precip_waterYear.head()

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5,year,month
Date,,,,,,,,,,,,
1950-01-01 00:00:00+00:00,0.820687,0.255154,0.780021,12.707801,0.402716,16.743221,0.000000,0.000000,0.00524,0.0,1950,1
1950-01-02 00:00:00+00:00,2.274546,0.363442,2.095610,4.049003,0.209240,6.724862,0.000000,0.247158,0.00000,0.0,1950,1
1950-01-03 00:00:00+00:00,18.358842,1.161089,0.587093,0.000000,0.234423,1.072030,0.035088,12.195158,0.29759,0.0,1950,1
1950-01-04 00:00:00+00:00,11.355387,0.598111,0.109596,0.000000,0.004274,4.068159,2.294689,8.228564,0.00000,0.0,1950,1
1950-01-05 00:00:00+00:00,5.819963,2.689298,0.000000,0.002790,0.000000,0.557955,0.021407,8.127099,0.00000,0.0,1950,1


In [16]:
#Water Year goes form Oct to Sep. 
precip_waterYear.loc[precip_waterYear.month >= 10, 'year'] = precip_waterYear.year+1

In [17]:
precip_waterYear = precip_waterYear[(precip_waterYear.month == 10) | (precip_waterYear.month == 11) | (precip_waterYear.month == 12) | (precip_waterYear.month == 1)|(precip_waterYear.month == 2)|(precip_waterYear.month == 3)| (precip_waterYear.month == 4)]

In [18]:
precip_waterYear_agg = precip_waterYear.groupby(['year']).sum()

In [19]:
precip_waterYear_agg.head()

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5,month
year,,,,,,,,,,,
1950,386.224195,304.764618,269.262512,331.787622,310.457039,96.221033,133.493195,245.055793,228.779019,245.645494,300
1951,255.641156,547.968106,342.623003,393.106415,618.109705,474.883421,380.378060,304.966021,138.134733,271.214252,1312
1952,264.286351,418.920579,257.327391,394.385525,360.694229,383.974226,350.163423,420.007698,484.773739,661.443601,1314
1953,199.394419,148.477074,542.303523,259.398111,411.901894,631.518311,273.329674,417.551934,442.089188,261.555926,1312
1954,487.843192,275.867863,374.595806,400.750940,358.262767,531.180880,339.518572,525.720259,747.385136,353.416842,1312


In [20]:
## Historical Data
prHist_waterYear_agg = precip_waterYear_agg.loc['1961':'1990']

In [21]:
upper_tercile = prHist_waterYear_agg.quantile(0.666)

In [22]:
upper_tercile

GCM
ACCESS1-0      453.144580
CCSM4          451.857365
CESM1-BGC      423.484069
CMCC-CMS       406.800232
CNRM-CM5       409.439454
CanESM2        428.677247
GFDL-CM3       438.463013
HadGEM2-CC     387.735215
HadGEM2-ES     422.745610
MIROC5         433.060495
month         1312.000000
Name: 0.666, dtype: float64

In [23]:
yrs_above_upper_tercile_pr = precip_waterYear_agg[precip_waterYear_agg > upper_tercile]

In [24]:
yrs_above_upper_tercile_pr.head()

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5,month
year,,,,,,,,,,,
1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951,NaN,547.968106,NaN,NaN,618.109705,474.883421,NaN,NaN,NaN,NaN,NaN
1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420.007698,484.773739,661.443601,1314.0
1953,NaN,NaN,542.303523,NaN,411.901894,631.518311,NaN,417.551934,442.089188,NaN,NaN
1954,487.843192,NaN,NaN,NaN,NaN,531.180880,NaN,525.720259,747.385136,NaN,NaN


### To find years below bottom decile (Sep-Dec)

In [25]:
precip.head()

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5,year,month
Date,,,,,,,,,,,,
1950-01-01 00:00:00+00:00,0.820687,0.255154,0.780021,12.707801,0.402716,16.743221,0.000000,0.000000,0.00524,0.0,1950,1
1950-01-02 00:00:00+00:00,2.274546,0.363442,2.095610,4.049003,0.209240,6.724862,0.000000,0.247158,0.00000,0.0,1950,1
1950-01-03 00:00:00+00:00,18.358842,1.161089,0.587093,0.000000,0.234423,1.072030,0.035088,12.195158,0.29759,0.0,1950,1
1950-01-04 00:00:00+00:00,11.355387,0.598111,0.109596,0.000000,0.004274,4.068159,2.294689,8.228564,0.00000,0.0,1950,1
1950-01-05 00:00:00+00:00,5.819963,2.689298,0.000000,0.002790,0.000000,0.557955,0.021407,8.127099,0.00000,0.0,1950,1


In [26]:
precip = precip[(precip.index.month == 9) | (precip.index.month == 10)| (precip.index.month == 11) | (precip.index.month == 12) ]

In [27]:
precip_agg = precip.groupby(precip.index.year).sum()

In [28]:
# histocial
pr_hist_agg = precip_agg.loc['1961':'1990']

In [29]:
bottom_decile = pr_hist_agg.quantile(0.10)

In [30]:
bottom_decile

GCM
ACCESS1-0         78.015861
CCSM4             56.576058
CESM1-BGC         54.746294
CMCC-CMS          71.406326
CNRM-CM5          85.658822
CanESM2           71.936845
GFDL-CM3          51.971301
HadGEM2-CC        71.212778
HadGEM2-ES        69.433308
MIROC5            61.538559
year          239687.800000
month           1282.000000
Name: 0.1, dtype: float64

In [31]:
yrs_below_bottom_decile = precip_agg[precip_agg < bottom_decile]

In [32]:
yrs_below_bottom_decile.head(10)

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5,year,month
Date,,,,,,,,,,,,
1950,NaN,NaN,NaN,NaN,NaN,58.054414,NaN,67.917475,20.72873,NaN,237992.0,NaN
1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238114.0,NaN
1952,NaN,36.842363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238236.0,NaN
1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238358.0,NaN
1954,NaN,NaN,27.624425,NaN,NaN,53.165097,NaN,NaN,NaN,NaN,238480.0,NaN
1955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.830511,NaN,NaN,238602.0,NaN
1956,NaN,NaN,NaN,NaN,46.793088,NaN,NaN,58.369476,NaN,NaN,238724.0,NaN
1957,NaN,NaN,NaN,NaN,65.997111,NaN,NaN,63.390089,NaN,NaN,238846.0,NaN
1958,NaN,NaN,NaN,NaN,NaN,14.147471,NaN,NaN,NaN,NaN,238968.0,NaN


## TMax

In [33]:
tmax.head()

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5
Date,,,,,,,,,,
1950-01-01 00:00:00+00:00,54.161400,52.582886,57.369904,57.100021,65.326645,59.927246,45.232979,54.752190,54.921764,53.597748
1950-01-02 00:00:00+00:00,57.349358,57.304207,58.465569,51.715736,63.081421,62.757103,44.550945,46.655376,47.753407,54.462921
1950-01-03 00:00:00+00:00,60.840263,53.671959,58.243370,45.774548,62.672344,57.619621,45.793610,53.948376,52.154366,53.432625
1950-01-04 00:00:00+00:00,56.377785,55.651695,58.438213,47.260010,61.072571,58.939522,46.588909,55.525520,52.648308,55.889771
1950-01-05 00:00:00+00:00,55.296947,59.364693,52.217480,41.186546,58.839653,48.802322,50.458183,54.587173,53.474316,56.541973


In [34]:
# To find top decile of may to Sep

tmax_mayToSep= tmax[(tmax.index.month == 5) | (tmax.index.month == 6)| (tmax.index.month == 7) | (tmax.index.month == 8)|(tmax.index.month == 9)]

In [35]:
# To find extreme hot of Oct to Dec
tmax_octToDec = tmax[(tmax.index.month == 10)|(tmax.index.month == 11) |(tmax.index.month == 12)]

In [36]:
tmax_mayToSep_agg = tmax_mayToSep.groupby(tmax_mayToSep.index.year).sum()

In [37]:
tmax_octToDec_agg = tmax_octToDec.groupby(tmax_octToDec.index.year).sum()

In [38]:
tmax_mayToSep_agg_hist = tmax_mayToSep_agg.loc['1961':'1990']
tmax_octToDec_agg_hist = tmax_octToDec_agg.loc['1961':'1990']

### top decile of may to Sep

In [39]:
top_decile = tmax_mayToSep_agg_hist.quantile(0.90)

In [40]:
top_decile

GCM
ACCESS1-0     13319.224264
CCSM4         13157.191199
CESM1-BGC     13162.968765
CMCC-CMS      13459.554821
CNRM-CM5      13180.394473
CanESM2       13099.571124
GFDL-CM3      13293.150006
HadGEM2-CC    13362.124426
HadGEM2-ES    13229.408656
MIROC5        13350.275755
Name: 0.9, dtype: float64

In [41]:
extreme_hot = tmax_octToDec_agg_hist.quantile(0.98)

In [42]:
type(tmax_mayToSep_agg)

pandas.core.frame.DataFrame

In [43]:
yrs_above_top_decile = tmax_mayToSep_agg[tmax_mayToSep_agg > top_decile]

In [44]:
yrs_above_extreme_hot = tmax_octToDec_agg[tmax_octToDec_agg> extreme_hot]

In [45]:
yrs_above_top_decile.head()

GCM,ACCESS1-0,CCSM4,CESM1-BGC,CMCC-CMS,CNRM-CM5,CanESM2,GFDL-CM3,HadGEM2-CC,HadGEM2-ES,MIROC5
Date,,,,,,,,,,
1950,NaN,NaN,NaN,NaN,NaN,NaN,13697.762218,13400.124313,NaN,NaN
1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952,NaN,NaN,NaN,NaN,13184.302856,NaN,NaN,NaN,NaN,NaN
1953,13353.736359,13389.194626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954,NaN,NaN,13194.725418,NaN,NaN,NaN,NaN,NaN,NaN,13356.852222


### Filtered Data

In [46]:
# yrs_above_upper_tercile_pr
# yrs_below_bottom_decile
# yrs_above_top_decile
# yrs_above_extreme_hot

In [47]:
filtered = pd.concat([yrs_above_upper_tercile_pr, yrs_below_bottom_decile, yrs_above_top_decile, yrs_above_extreme_hot], axis = 1 )

In [48]:
models

array(['ACCESS1-0', 'CCSM4', 'CESM1-BGC', 'CMCC-CMS', 'CNRM-CM5',
       'CanESM2', 'GFDL-CM3', 'HadGEM2-CC', 'HadGEM2-ES', 'MIROC5'],
      dtype=object)

In [49]:
access1_0 = []
ccsm4 = []
cesm1_bgc = []
cmcc_cms = []
cnrm_cm5 = []
canesm2 = []
gfdl_cm3 = []
hadgem2_cc = []
hadgem2_es = []
miroc5 = []
dataModels = [access1_0, ccsm4, cesm1_bgc,  cmcc_cms, cnrm_cm5, canesm2, gfdl_cm3, hadgem2_cc, hadgem2_es, miroc5 ]

In [50]:
for model, i in zip(models, range(0, 10)):
    dataModels[i] = pd.DataFrame({'agg_pr_OctToApril' : yrs_above_upper_tercile_pr[model],
                             'agg_pr_SepTodec' : yrs_below_bottom_decile[model],
                              'agg_tmax_MayToSep':yrs_above_top_decile[model],
                              'agg_tmax_OctToDec': yrs_above_extreme_hot[model]     
                             })

In [51]:
dataModels[5].head()

,agg_pr_OctToApril,agg_pr_SepTodec,agg_tmax_MayToSep,agg_tmax_OctToDec
1950,NaN,58.054414,NaN,NaN
1951,474.883421,NaN,NaN,NaN
1952,NaN,NaN,NaN,NaN
1953,631.518311,NaN,NaN,NaN
1954,531.180880,53.165097,NaN,NaN


In [52]:
for i in dataModels:
    i.dropna(inplace = True)

In [53]:
dataModels[7].head()

,agg_pr_OctToApril,agg_pr_SepTodec,agg_tmax_MayToSep,agg_tmax_OctToDec
2032,473.652426,49.232301,13807.705070,6650.148552
2033,450.548989,61.024016,13714.308502,6671.491020
2074,553.405663,70.619335,13934.685188,7117.422752
2084,437.478828,32.347589,14757.682121,6912.579987
2091,531.712170,50.612200,14363.967270,7078.657188


In [54]:
for i in dataModels:
    print(i.index.values)

[2061 2099]
[2088 2092]
[2048 2058]
[2040 2066 2078 2079 2083 2089 2095]
[2004 2036 2039 2051 2057 2065 2070 2080 2082 2087 2089 2095 2100]
[2032 2033 2049 2091 2095]
[2071 2077 2091 2093 2095]
[2032 2033 2074 2084 2091 2098]
[2033 2059 2078 2080 2081 2096 2098]
[2072]


In [55]:
models

array(['ACCESS1-0', 'CCSM4', 'CESM1-BGC', 'CMCC-CMS', 'CNRM-CM5',
       'CanESM2', 'GFDL-CM3', 'HadGEM2-CC', 'HadGEM2-ES', 'MIROC5'],
      dtype=object)